## Hmm.. Lets Ben:

As an user you are suppose to modify just the first cell. and run the entire notebook. Don't worry about repeating task or subtasks during network problems and any other failures. The code is written in a way to not repeat task when failure occurs and you want to initiate the code form the failed point.

* input_csv_path: Is your path to csv file with property address. Note this file has a specific format. If the format is not met then the cleaning process would through an error.
* which_run: This can be any name but should be unique to previous run. If not unique then it will overwrite the output of any previous run with the same name. (example: 'may_31_2018'). This is improtant becasue you can find all the relevant statistics for your run int subfolder creates with the provided name.
* image_type: Represents the image type you would like to run.
* filter_conditions: This parameter may look weird, but we have it in this way to sync it with the apache Airflow variable declaration. You just have to change the third value after the ":".

and so on ......

In [16]:
from datetime import datetime, timedelta
from jobs.data_load import dump_aerial, dump_aerial_cropped, dump_overlaid, create_csv_file_for_input
from jobs.batch_create import prepare_batches, remove_batches
from jobs.train_cv_test import test_new, predictions




input_csv_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/house_metadata_nw.csv'
which_run = 'new_test'
image_type = 'aerial_cropped'
use_checkpoint_of_run = 'sam_new'
prediction_threshold = 0.68
filter_conditions = 'last_reviewed_ts : bool : True \n\
from_year : int : 2015 \n\
to_year : int : 2016 \n\
from_month : int : 6 \n\
to_month : int : 7 \n\
which_city : str : chicago \n\
use_improvement_lvl : bool : True \n\
max_num_records : int : 264'

which_net = 'resnet'
batch_size = 128
proportion_cv_data = 0.05
proportion_test_data = 0.05

# print(filter_conditions)  
# print ('')
# Parse Variables
cond_dict = {}
filter_conditions = filter_conditions.split('\n')
# print (filter_conditions)
for conds in filter_conditions:
#     print (conds,'\n')
    k, func, v = conds.split(':')
#     print (k, func, v)
    if func.strip() == 'None':
        cond_dict[k.strip()] = None
    elif func.strip() == 'bool':
        cond_dict[k.strip()] = bool(v.strip())
    elif func.strip() == 'int':
        cond_dict[k.strip()] = int(v.strip())
    elif func.strip() == 'float':
        cond_dict[k.strip()] = float(v.strip())
    elif func.strip() == 'str':
        cond_dict[k.strip()] = str(v.strip())

In [17]:
cond_dict

{'from_month': 6,
 'from_year': 2015,
 'last_reviewed_ts': True,
 'max_num_records': 264,
 'to_month': 7,
 'to_year': 2016,
 'use_improvement_lvl': True,
 'which_city': 'chicago'}

## CLEAN RECORDS FROM THE CSV FILE.

In [21]:
param_dict = dict(
          input_csv_path=input_csv_path,
          which_run=which_run,
          img_type= image_type,
          cond_dict=cond_dict
)

create_csv_file_for_input(params=param_dict)

/Users/sam/App-Setup/anaconda/envs/anaconda35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Shape: Metadata table (220364, 18)
Data shape  = (220364, 18)
Applying conditions: from_year 2015 - to_year 2015
Data shape  = (21683, 18)
Applying conditions: from_month 6 - to_month 6
Data shape  = (11175, 18)
Applying conditions: city: chicago
Data shape  = (9267, 18)
Applying conditions: improvement_lvl: True
Data shape land = (4265, 18), house = (4897, 18)
Applying conditions: max_num_records: 264
Shape: Land Property table (132, 18)
Shape: House Property table (132, 18)


'COMPLETE'

## FETCH AERIAL IMAGES

In [23]:
param_dict=dict(
      which_run=which_run,
      img_type= image_type
)

dump_aerial(params=param_dict)

'COMPLETE'

## DUMP AERIAL CROPPED IMAGES

In [24]:
param_dict=dict(
      which_run=which_run,
      img_type= image_type
)

if image_type == 'aerial_cropped':
    dump_aerial_cropped(params=param_dict)

Initiating Run for scoop scoop_738_768 : DATA SHAPE (93246, 13)
Total images to be processed are  (32, 9)
Initiating Run for scoop scoop_768_798 : DATA SHAPE (94331, 13)
Total images to be processed are  (28, 9)
Initiating Run for scoop scoop_798_848 : DATA SHAPE (80697, 13)
Total images to be processed are  (19, 9)
Initiating Run for scoop scoop_848_908 : DATA SHAPE (90177, 13)
Total images to be processed are  (28, 9)
Initiating Run for scoop scoop_908_938 : DATA SHAPE (92080, 13)
Total images to be processed are  (6, 9)
Initiating Run for scoop scoop_938_968 : DATA SHAPE (91240, 13)
Total images to be processed are  (3, 9)
Initiating Run for scoop scoop_968_028 : DATA SHAPE (86448, 13)
Total images to be processed are  (8, 9)


'COMPLETE'

## DUMP OVERLAID IMAGES

In [25]:
param_dict=dict(
      which_run=which_run,
      img_type= image_type
)

if image_type == 'overlaid':
    dump_overlaid(params=param_dict)

## TEST ON NEW IMAGES:

In [27]:
param_dict = dict(
      which_run=which_run,
      img_type=image_type,
      use_checkpoint_for_run=use_checkpoint_of_run,
      use_checkpoint_for_imageType=image_type,
      which_net=which_net
)

test_new(params=param_dict)

Test Graphs: RESNET
Learning Rate: Initial:  0.0005
INFO:tensorflow:Restoring parameters from /Users/sam/All-Program/App-DataSet/HouseClassification/checkpoints/sam_new/aerial_cropped/resnet/resnet_epoch_29_batch_89
Batch path /Users/sam/All-Program/App-DataSet/HouseClassification/batch_data/new_test/aerial_cropped, batch_names: ['batch_17', 'batch_6', 'batch_13', 'batch_2', 'batch_12', 'batch_3', 'batch_16', 'batch_7', 'batch_8', 'batch_9', 'batch_11', 'batch_0', 'batch_15', 'batch_4', 'batch_14', 'batch_5', 'batch_10', 'batch_1']
Test Graphs: RESNET
Learning Rate: Initial:  0.0005
INFO:tensorflow:Restoring parameters from /Users/sam/All-Program/App-DataSet/HouseClassification/checkpoints/sam_new/aerial_cropped/resnet/resnet_epoch_29_batch_99
Batch path /Users/sam/All-Program/App-DataSet/HouseClassification/batch_data/new_test/aerial_cropped, batch_names: ['batch_17', 'batch_6', 'batch_13', 'batch_2', 'batch_12', 'batch_3', 'batch_16', 'batch_7', 'batch_8', 'batch_9', 'batch_11', 'bat

'COMPLETE'

## MAKE FINAL PREDICTIONS

In [28]:
param_dict = dict(
        which_run=which_run,
        img_type=image_type,
        use_checkpoint_for_run=use_checkpoint_of_run,
        use_checkpoint_for_imageType=image_type,
        which_net=which_net,
        use_checkpoint_for_prediction='all',
        classification_threshold=prediction_threshold
)

predictions(params=param_dict)

Checkpoint LISTS .. %s ['resnet_epoch_30_batch_9', 'resnet_epoch_29_batch_89', 'resnet_epoch_29_batch_116', 'resnet_epoch_29_batch_99', 'resnet_epoch_29_batch_109']
pred_stats.shape = (11870, 6), meta_stats.shape = (2374, 5)
The accuracy using predictions from multiple checkpoint is:  0.887952822241


'COMPLETE'

## REMOVE BATCHES

In [30]:
params = dict(
        which_run=which_run,
        img_type=image_type
)

remove_batches(params=param_dict)